<a href="https://colab.research.google.com/github/AnDDoanf/LLM-repo/blob/master/RAG_mistral7b_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
%%capture
!pip install transformers bitsandbytes accelerate langchain langchain_community playwright html2text sentence-transformers faiss-gpu
!playwright install

In [2]:
%%capture
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
from transformers import BitsAndBytesConfig
from torch import cuda, bfloat16

model_name='mistralai/Mistral-7B-Instruct-v0.1'

model_config = transformers.AutoConfig.from_pretrained(
    model_name,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

In [3]:
inputs_not_chat = tokenizer.encode_plus("[INST] Tell me about fantasy football? [/INST]", return_tensors="pt")['input_ids'].to('cuda')

generated_ids = model.generate(inputs_not_chat,
                               max_new_tokens=1000,
                               do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)

In [4]:
decoded

['<s> [INST] Tell me about fantasy football? [/INST] Fantasy football is a popular game in which players simulate the management of a team of real-life football players. The game usually involves creating a virtual roster of players from different real-world teams and using various rules and strategies to compete against other players online in simulated games. Participants also typically need to keep track of things like injuries, player statistics, and real-time match updates to make informed decisions about their lineup. There are various leagues and rules that can be used to make the game more challenging or dynamic, and participants can often engage in friendly banter and competition with other fantasy football enthusiasts. Overall, fantasy football is a fun and engaging way for people to get involved in the world of football and experience the game in a new way.</s>']

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.vectorstores import FAISS
import nest_asyncio
from langchain.embeddings import HuggingFaceEmbeddings
nest_asyncio.apply()

articles = ["https://printer.gotquestions.net/GeneratePF?articleId=" + str(index) for index in range(200)]

# Scrapes the blogs above
loader = AsyncChromiumLoader(articles)
docs = loader.load()

# Converts HTML to plain text
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Chunk text
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

# Load chunked documents into the FAISS index
db = FAISS.from_documents(chunked_documents,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [46]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=300,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on the provided context.
If you do not know, say you don't know.
Do cite your sources.
[INST]
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

retriever = db.as_retriever()

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [48]:
rag_chain.invoke("what is Luciferianism?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


{'context': [Document(page_content='**Question: "What is Luciferianism?"**', metadata={'source': 'https://printer.gotquestions.net/GeneratePF?articleId=100'}),
  Document(page_content='**Answer:** One type of Luciferianism is the worship or reverence of Lucifer\nas a deity. Such religion is related to Satanism—though it attempts to\nemphasize the more “positive” aspects of Lucifer. Another type of\nLuciferianism is nontheistic and views Lucifer as nothing more than a symbol\nof mankind’s quest for wisdom and enlightenment.  \n  \nThe name “Lucifer” comes from a translation of Isaiah 14:12. It literally\nmeans “bright star, shining star, or morning star.” Most scholars see this as\na description of Satan before his rebellion against God. Passages like Isaiah\n14 and Ezekiel 28 teach that Satan was created the highest, most beautiful of\nthe angels, but that his pride and desire for God’s own throne resulted in his\nbeing cast out of heaven and being given the name “Satan” (meaning\n“adv